## Calculate OLR versus $T_s$ with radiative-convective atmospheres

### Import things
This assumes that SOCRATES is installed at `AGNI/socrates/`. Modify this as required.

In [1]:
# Set directory
ROOT_DIR = abspath(joinpath(pwd(),"../"))
ENV["RAD_DIR"] = joinpath(ROOT_DIR,"socrates")

"/Users/nichollsh/Projects/AGNI/socrates"

In [2]:
# Import system packages
using Printf
using Plots

# Import AGNI
using AGNI
import AGNI.atmosphere as atmosphere
import AGNI.solver as nl
import AGNI.dump as dump
import AGNI.plotting as plotting

# Disable logging from AGNI module
AGNI.setup_logging("",false)

### Define atmosphere parameters

In [3]:
# Configuration options
instellation    = 1360.0  # Solar flux [W m-2]
gravity         = 9.81
radius          = 6.37e6
nlev_centre     = 40
p_surf          = 270.0     # bar
t_surf          = 1000.0
p_top           = 1e-5      # bar 
mole_fractions  = Dict([("H2O", 1.0)])
condensates     = ["H2O"]

spectral_file = joinpath(ROOT_DIR,"res/spectral_files/Frostflow/16/Frostflow.sf")
star_file     = joinpath(ROOT_DIR,"res/stellar_spectra/sun.txt");


In [4]:
# Create output directory
output_dir = "/tmp/agni_notebook/"
rm(output_dir,force=true,recursive=true)
mkdir(output_dir);

### Initialise the atmosphere

In [5]:
# Setup atmosphere
atmos = atmosphere.Atmos_t()
atmosphere.setup!(atmos, ROOT_DIR, output_dir, 
                        spectral_file,
                        instellation, 0.375, 0.0, 48.19,
                        t_surf,  
                        gravity, radius,
                        nlev_centre, p_surf, p_top,
                        mf_dict=mole_fractions,
                        condensates=condensates,
                        flag_gcontinuum=true,
                        thermo_functions=false
                        )
atmosphere.allocate!(atmos, star_file)

[ INFO  ] Composition set by dict 
[ INFO  ]     added gas H2O 
[ INFO  ] Inserting stellar spectrum 


### Run the model for a high-temperature case
This provides an initial guess from which later simulations will be initialised

In [ ]:
atmos.tmp_surf = 3050.0
atmos.tmpl[:] .= 3000
atmos.tmp[:] .= 3000;

In [ ]:
solver_success = nl.solve_energy!(atmos, 
                                    sol_type=1,         # Conserve energy, but with fixed surface temperature
                                    sens_heat=true,    # Do not include sensible heat transport
                                    latent=true,        # Include condensation
                                    method=1,           # Use the Newton-Raphson method
                                    dx_max=400.0,       # Allow large step sizes because of the poor initial guess
                                    linesearch=true ,   # Enable Linesearch
                                    save_frames=false, modplot=1, # disable plotting 
                                    conv_atol=0.1
                                    )
println("Solver success? $solver_success")

In [ ]:
out = joinpath(output_dir, "base.ncdf")
dump.write_ncdf(atmos, out);

Plot this profile, to check that it looks reasonable...

In [ ]:
plotting.plot_pt(atmos, "")

This shows convection from the surface and condensation in the upper atmosphere. This is because there's large upward energy flux as the planet attempts to cool down under the modest instellation.     

### Run the model for a range of surface temperatures

In [ ]:
num_loops = 5
tmp_arr = range(start=2000.0, stop=1500.0, length=num_loops)
atm_arr = atmosphere.Atmos_t[]

for (i,tmp) in enumerate(tmp_arr)
    # Set new surface temperature 
    println("Running model for tmp_surf = $tmp")
    atmos.tmp_surf = tmp 

    # Help solver by changing initial guess
    clamp!(atmos.tmpl, 0.0, tmp)
    clamp!(atmos.tmp, 0.0, tmp)

    # Run model
    solver_success = nl.solve_energy!(atmos, 
                                    sol_type=1,         
                                    sens_heat=true,   
                                    latent=true,        
                                    method=1,           
                                    dx_max=300.0,        # Smaller steps
                                    linesearch=true ,   
                                    save_frames=false, modplot=1,
                                    conv_atol=0.1      # Tighter tolerance on solver 
                                    )

    # Store result
    push!(atm_arr, deepcopy(atmos))

    println("---------------------------------")
end 
println("Done!")

### Make plot of temperature profiles

In [ ]:
plt = plot(framestyle=:box, size=(500,400), dpi=300)

for this_atm in atm_arr
    plot!(plt, this_atm.tmpl, this_atm.pl, label=@sprintf("%.1f",this_atm.tmp_surf))
end 

xlabel!(plt, "Temperature [K]")
ylabel!(plt, "Pressure [bar]")
yflip!(plt)
yaxis!(plt, yscale=:log10)
display(plt)

### Make plot of OLR vs $T_s$

In [ ]:
plt = plot(framestyle=:box, size=(700,400))

olr_arr = Float64[]
for this_atm in atm_arr
    push!(olr_arr, this_atm.flux_u_lw[1])
end 

hline!(plt, [281.0], label="S-N limit", color="black", linestyle=:dash)

# Plot results
plot!(plt, tmp_arr, olr_arr, label="AGNI result", color="red")
scatter!(plt, tmp_arr, olr_arr, label="", color="red")

xlabel!(plt, "Surface temperature [K]")
ylabel!(plt, "OLR [W m⁻²]")
display(plt)